# ECSE 415 - Final Project

## 0 Setup

We start by importing the relevant libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

from typing import Generator
from cv2.typing import MatLike

Next, we define a `PATH` variable that points to the repository. Every memory access in the code will be relative to that variable.

We also define a few functions that will be useful throughout the code:
- `showImages`: This function displays a grid of images with their titles.
- `showGSImages`: This function displays a grid of grayscale images with their titles.

In [2]:
PATH = "./"

np.set_printoptions(edgeitems=24)

def showImages(images, titles):
    '''This function takes a 2D array of images and titles and displays them in the notebook in color'''
    rows = len(images)
    cols = len(images[0])
    fig = plt.figure(figsize=(cols * 7 if cols > 1 else 12, rows * 5))

    # Add every picture in the plot
    for i in range(rows):
        for j in range(cols):
            if images[i][j] is None: continue
            fig.add_subplot(rows, cols, i * cols + j + 1)
            plt.imshow(images[i][j][:,:,::-1])
            plt.axis('off')
            plt.title(titles[i][j])
    
    plt.show()


def showGSImages(images, titles, cmap=[]):
    '''This function takes a 2D array of images and titles and displays them in the notebook in grayscale'''
    rows = len(images)
    cols = len(images[0])
    fig = plt.figure(figsize=(cols * 7 if cols > 1 else 12, rows * 5))

    # Add every picture in the plot
    for i in range(rows):
        for j in range(cols):
            if images[i][j] is None: continue
            fig.add_subplot(rows, cols, i * cols + j + 1)
            plt.imshow(images[i][j], cmap=cmap[i][j] if len(cmap) != 0 else 'gray')
            plt.axis('off')
            plt.title(titles[i][j])
    
    plt.show()


# 1 Video Reader

We define a `VideoReader` class that helps with reading video from the filesystem.

It is based on OpenCV's `VideoCapture` class, and helps with reading the frames of a video, as well as getting information such as the frame dimension and the FPS value.

The main purpose of this class is to wrap the functionalities needed to read every nth frame of a video using the `VideoReader.frames(n)` function.

In [3]:
class VideoReader():
    """
    A class used to read videos from the filesystem.
    
    ---

    Attributes
    ----------
    `filename` : `str`
        A string pointing to the path of the video in the file system.
    
    `cap` : `VideoCapture`
        The underlying OpenCV `VideoCapture` that helps read the video file.

    `framewidth` : `int`
        An integer indicating the width of the frames in the video.

    `frameheight` : `int`
        An integer indicating the height of the frames in the video.
    
    `fps` : `float`
        A floating-point number indicating the framerate of the video.

    ---

    Methods
    -------
    `getInfo() -> (int, int, float)`
        Get basic information about the video: frame width, frame height, frame rate.

    `frames(freq: int) -> Generator[MatLike, None, None]`
        Generator function that yields every nth frame.

    `release()`
        Video capture release.
    """
    
    def __init__(self, filename: str):
        # Open video
        self.cap = cv2.VideoCapture(filename)

        # Ensure video was correctly opened or raise exception
        if(self.cap.isOpened() == False):
            raise SystemExit("Error opening video file!")
        
        # Get basic information of video
        self.frame_width = int(self.cap.get(3))
        self.frame_height = int(self.cap.get(4))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
    
    
    def getInfo(self) -> (int, int, float):
        """
        Get basic information about the video: frame width, frame height, frame rate.

        ---

        Returns
        -------
        `int`
            Frame width

        `int`
            Frame height
            
        `float`
            Frame rate
        """
        return self.frame_width, self.frame_height, self.fps
    

    def frames(self, freq: int) -> Generator[MatLike, None, None]:
        """
        Generator function that yields every nth frame of the video.

        ---

        Parameters
        ----------
        `freq`: `int`
            Integer representing the number of frames to skip before yielding the next one.
        
        ---

        Yields
        ------
        `MatLike`
            Next frame of the video.
        """
        i = 0
        while(self.cap.isOpened()):
            ret, frame = self.cap.read()
            
            # If end of video is reached, exit the generator loop
            if ret == False: break

            i += 1
            # Skip frame if its index is not a multiple of freq
            if i % freq != 0: continue

            yield frame
    
    
    def release(self):
        """
        Function that releases the `VideoCapture` element that helped read the video.
        """
        self.cap.release()

In [4]:
class BoundingBox():
    """
    A class that is used to represent a bounding box detected by object detector models.

    ---

    Attributes
    ----------
    `x`: `float`
        Float representing the x-coordinate of the top-left corner of the bounding box.
    
    `y`: `float`
        Float representing the y-coordinate of the top-left corner of the bounding box.
    
    `w`: `float`
        Float representing the width of the bounding box.
    
    `h`: `float`
        Float representing the height of the bounding box.
    
    `clazz`: `int`
        Class ID of the detected object.
    
    `confidence`: `float`
        Float between 0 and 1 representing the classification confidence of the object in the
        bounding box.
    
    `center_x`: `float`
        Float representing the x-coordinate of the center point of the bounding box.
    
    `center_y`: `float`
        Float representing the y-coordinate of the center point of the bounding box.
    
    ---

    Methods
    -------
    `get_distance(other: BoundingBox)`
        This function computes the distance between the center point of this bounding box and the
        center point of another bounding box.
    
    `get_top_left_corner() -> (int, int)`
        This function returns a tuple holding the rounded coordinates of the top-left corner of the
        bounding box.
    
    `get_lower_right_corner() -> (int, int)`
        This function returns a tuple holding the rounded coordinates of the lower-right corner of
        the bounding box.
    """
    
    def __init__(self, x: float, y: float, w: float, h: float, clazz: int, confidence: float):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.clazz = clazz
        self.confidence = confidence

        # Compute center coordinates of the bounding box
        self.center_x = x + w / 2
        self.center_y = y + h / 2
    

    def get_distance(self, other: "BoundingBox") -> float:
        """
        This function computes the distance between the center point of this bounding box and the
        center point of another bounding box.

        ---

        Parameters
        ----------
        `other`: `BoundingBox`
            The other bounding box with which to compute the distance.
        
        ---

        Returns
        -------
        `float`
            Distance between this bounding box and the other.
        """
        return np.sqrt((self.center_x - other.center_x) ** 2 + (self.center_y - other.center_y) ** 2)


    def get_top_left_corner(self) -> (int, int):
        """
        This function returns a tuple holding the rounded coordinates of the top-left corner of the
        bounding box.

        ---

        Returns
        -------
        `(int, int)`
            Tuple of the rounded coordinates (x, y) of the top-left corner of the bounding box.
        """
        return (round(self.x), round(self.y))
    

    def get_lower_right_corner(self) -> (int, int):
        """
        This function returns a tuple holding the rounded coordinates of the lower-right corner of 
        the bounding box.

        ---

        Returns
        -------
        `(int, int)`
            Tuple of the rounded coordinates (x, y) of the lower-right corner of the bounding box.
        """
        return (round(self.x + self.w), round(self.y + self.h))

In [5]:
class Object():
    """
    A class that represents an object in the video.

    ---

    Attributes
    ----------
    `id`: `int`
        Unique ID of the object in the video.
    
    `bounding_box`: `BoundingBox`
        The bounding box of the object in the video. This is updated every frame.
    
    `absent_frames`: `int`
        The number of frames since the object disappeared from the scene.
    """

    def __init__(self, id: int, bounding_box: BoundingBox):
        self.id = id
        self.bounding_box = bounding_box
        self.absent_frames = 0


In [6]:
class Yolo:
    """
    A class that contains a YOLOv3 model pretrained on the COCO dataset.
    It wraps the functionalities to process a picture on this model and retrieve the bounding boxes
    of the detected and classified objects.

    ---

    Attributes
    ----------
    `yolov3`: `Net`
        YOLOv3 neural network pretrained on the COCO dataset.

    `classes`: `list[str]`
        List of classes that can be classified by the model.
    
    `colors`: `ndarray[Any, dtype[float64]]`
        Randomly-generated RGB tuples list where each tuple represents the color associated to a
        class. For the purposes of drawing the bounding boxes.
    
    ---

    Methods
    -------
    `process_frame(frame: MatLike) -> list[BoundingBox]`
        This function processes an image using the model and returns a list of the resulting
        bounding boxes.
    
    `draw_bounding_box(img: MatLike, obj: Object)`
        This function draws a given object and its associated bounding box on a given image.
    """
    
    def __init__(self):
        # Import Darknet Yolov3 CNN from file as well as weights from COCO pretraining
        self.yolov3 = cv2.dnn.readNet(PATH + "/data/yolov3.weights", PATH + "/data/yolov3.cfg")

        # Import COCO classes from file
        self.classes = []
        classes_file = open(PATH + "/data/yolov3.classes", "r")
        for l in classes_file.readlines():
            self.classes.append(l.strip())

        # Generate random colors for each class to draw bounding boxes
        self.colors = np.random.uniform(0, 255, size=(len(self.classes), 3))
    
    
    def process_frame(self, frame: MatLike) -> list[BoundingBox]:
        """
        This function processes an image using the model and returns a list of the resulting
        bounding boxes.

        ---

        Parameters
        ----------
        `frame`: `MatLike`
            Image to be processed by the model.
        
        ---

        Returns
        -------
        `list[BoundingBox]`
            The list of bounding boxes found by the model.
        """
        frame_blob = cv2.dnn.blobFromImage(frame, 0.00392, (416,416), (0,0,0), True, crop=False)

        # Execute neural network with image as input
        self.yolov3.setInput(frame_blob)
        output = self.yolov3.forward()

        class_ids = []
        confidences = []
        boxes = []

        CONF_THRESH = 0.5
        # Extract data from output matrix
        for o in output:
            # Get predicted class and confidence for bounding box
            scores = o[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Consider bounding box only if confidence is above threshold value
            if confidence > CONF_THRESH:
                center_x = int(o[0] * frame.shape[1])
                center_y = int(o[1] * frame.shape[0])
                w = int(o[2] * frame.shape[1])
                h = int(o[3] * frame.shape[0])
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

        # Apply non-max suppression
        NMS_THRESH = 0.4
        indices = cv2.dnn.NMSBoxes(boxes, confidences, CONF_THRESH, NMS_THRESH)

        # Create the bounding box objects
        bounding_boxes = []
        for idx in indices:
            bounding_boxes.append(BoundingBox(boxes[idx][0], boxes[idx][1], boxes[idx][2], boxes[idx][3], class_ids[idx], confidences[idx]))

        return bounding_boxes


    # function to draw bounding box on the detected object with class name
    def draw_bounding_box(self, img: MatLike, obj: Object):
        """
        This function draws a given object and its associated bounding box on a given image.

        ---

        Parameters
        ----------
        `img`: `MatLike`
            Image on which to draw the object and its bounding box.
        
        `obj`: `Object`
            Object to draw on the image.
        """
        label = "{} {} (#{})".format(self.classes[obj.bounding_box.clazz], round(obj.bounding_box.confidence, 2), obj.id)
        color = self.colors[obj.bounding_box.clazz]
        cv2.rectangle(img, obj.bounding_box.get_top_left_corner(), obj.bounding_box.get_lower_right_corner(), color, 2)
        cv2.putText(img, label, (round(obj.bounding_box.x - 10), round(obj.bounding_box.y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

In [7]:
DISTANCE_THRESHOLD = 75
MAX_ABSENT_FRAMES = 5

class Objects():
    """
    A class that contains the objects present in the scene and the necessary logic to track objects
    across frames.

    ---

    Attributes
    ----------
    `idx`: `int`
        Integer iterator that tracks the next unique ID that can be given to a new object.
    
    `objects`: `dict[int, list[Object]]`
        Dictionary contianing lists of objects categorized by their class.
    
    ---

    Methods
    -------
    `add_frame(bbs: list[BoundingBox])`
        This function takes in the bounding boxes of the next frame and matches the existing objects
        to the new bounding boxes. It updates the the matched objects, creates new objects for new
        bounding boxes, and removes objects that no longer exist, when appropriate.
    
    `compute_distances(clazz: int, bbs: list[BoundingBox]) -> dict[tulpe[Object, BoundingBox], float]`
        This function computes all the distances between the objects in the class's dictionary that
        correspond to the input class ID, and the input bounding boxes.
    
    `match_bounding_boxes(distances: dict[tuple[Object, BoundingBox], float]) -> (list[tuple[Object, BoundingBox]], list[Object], list[BoundingBox])`
        This function finds the matches between the existing objects in the scene and the input
        bounding boxes.
    
    `next_id() -> int`
        This function increments `idx` and returns the new value. This serves when creating a new
        object.
    
    `get_all_objects() -> list[Object]`
        This function returns a list of all objects in the scene by flattening the dictionary
        values.
    """

    def __init__(self):
        self.idx = -1
        self.objects: dict[int, list[Object]] = dict({})


    def add_frame(self, bbs: list[BoundingBox]):
        """
        This function takes in the bounding boxes of the next frame and matches the existing objects
        to the new bounding boxes. It updates the the matched objects, creates new objects for new
        bounding boxes, and removes objects that no longer exist, when appropriate.

        ---

        Parameters
        ----------
        `bbs`: `list[BoundingBox]`
            The new bounding boxes found in the next frame.
        """
        # For each objects of a given class
        for c in self.objects.keys():
            # Find the matches with the new bounding boxes
            bbs_c = list(filter(lambda bb: bb.clazz == c, bbs))
            distances = self.compute_distances(c, bbs_c)
            matches, used_objs, used_bbs = self.match_bounding_boxes(distances)

            # Eliminate matches that excees the distance threshold
            for match in matches:
                if distances[match] > (match[0].absent_frames + 1) * DISTANCE_THRESHOLD:
                    used_objs.remove(match[0])
                    used_bbs.remove(match[1])
                    matches.remove(match)

            # Increment the absent_frames attribute of all objects that do not appear in the new
            # frame
            for obj in self.objects[c]:
                if not obj in used_objs:
                    obj.absent_frames += 1

                    if obj.absent_frames > MAX_ABSENT_FRAMES:
                        self.objects[c].remove(obj)
            
            # Create a new object for every bounding box that was not present in the previous frames
            for bb in bbs_c:
                if not bb in used_bbs:
                    self.objects[c].append(Object(self.next_id(), bb))

            # Update the bounding boxes of the matched objects
            for match in matches:
                match[0].bounding_box = match[1]
                match[0].absent_frames = 0
        
        # For every bounding box whose class was not previously present in the dictionary:
        # 1. Add a new list entry to the dictionary
        # 2. Create a new object for the bounding box
        # 3. Append the new object to the dictionary
        for bb in list(filter(lambda bb: (not bb.clazz in self.objects), bbs)):
            if bb.clazz not in self.objects: self.objects[bb.clazz] = []
            self.objects[bb.clazz].append(Object(self.next_id(), bb))


    def compute_distances(self, clazz: int, bbs: list[BoundingBox]) -> dict[tuple[Object, BoundingBox], float]:
        """
        This function computes all the distances between the objects in the class's dictionary that
        correspond to the input class ID, and the input bounding boxes.

        ---

        Parameters
        ----------
        `clazz`: `int`
            Class ID whose objects to get from the class's dictionary.
        
        `bbs`: `list[BoundingBox]`
            Bounding boxes to compute the distances with. These should also be of the class `clazz`.
        
        ---

        Returns
        -------
        `dict[tuple[Object, BoundingBox], float]
            Dictionary mapping each object-bounding box pair to the distance that separates them.
        """
        distances: dict[tuple[Object, BoundingBox], float] = dict({})

        # Compute distances between all objects and all bounding boxes
        for obj in self.objects[clazz]:
            for bb in bbs:
                distances[(obj, bb)] = obj.bounding_box.get_distance(bb)

        return distances


    def match_bounding_boxes(self, distances: dict[tuple[Object, BoundingBox], float]) -> (list[tuple[Object, BoundingBox]], list[Object], list[BoundingBox]):
        """
        This function finds the matches between the existing objects in the scene and the input
        bounding boxes.

        ---

        Parameters
        ----------
        `distances`: `dict[tuple[Object, BoundingBox], float]`
            Dictionary mapping each object-bounding box pair to the distance that separates them.
        
        ---

        Returns
        -------
        `list[tuple[Object, BoundingBox]]`
            List of object-bounding box pairs representing the matches. Each object and bounding box
            appears only once in the list.
        
        `list[Object]`
            List of matched objects.

        `list[BoundingBox]`
            List of matched bounding boxes.
        """
        # Sort the distances dictionary
        sorted_keys = list(sorted(distances, key=distances.get))

        mutex_keys = []
        used_objs = set([])
        used_bbs = set([])
        for k in sorted_keys:
            # If object or bounding box is already match, skip entry
            if k[0] in used_objs or k[1] in used_bbs: continue

            # Mark the matched object and bounding box as used
            used_objs.add(k[0])
            used_bbs.add(k[1])

            # Append the pair to the list of matches
            mutex_keys.append(k)

        return mutex_keys, used_objs, used_bbs


    def next_id(self) -> int:
        """
        This function increments `idx` and returns the new value. This serves when creating a new
        object.

        ---

        Returns
        -------
        `int`
            The unique ID to use for the new object.
        """
        self.idx += 1
        return self.idx
    

    def get_all_objects(self) -> list[Object]:
        """
        This function returns a list of all objects in the scene by flattening the dictionary
        values.

        ---

        Returns
        -------
        `list[Object]`
            The list of all objects in the scene.
        """
        return  [value for sublist in self.objects.values() for value in sublist]

In [8]:
# Create an instance of the model
yolo = Yolo()

# Create the video reader
video_in = VideoReader(PATH + "/data/mcgill_drive.mp4")

# Get basic information on video and print them
frame_width, frame_height, fps = video_in.getInfo()
print("Frame dimensions: ({}, {})".format(frame_width, frame_height))
print("FPS: {}".format(fps))

# Create the output video stream
out = cv2.VideoWriter(PATH + "/out/mcgill_drive.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width,frame_height))

# Create the object that holds all objects in the scene
objects = Objects()

# Loop over every frame
for frame in video_in.frames(1):
    # Process frame and process new bounding boxes for tracking
    bounding_boxes = yolo.process_frame(frame)
    objects.add_frame(bounding_boxes)
    
    # Draw each object on the frame picture
    for obj in objects.get_all_objects():
        yolo.draw_bounding_box(frame, obj)
    
    # Write the new frame picture with bounding boxes to the output video
    out.write(frame)


# Release video streams
video_in.release()
out.release()

Frame dimensions: (2562, 1440)
FPS: 29.97002997002997
